<a href="https://colab.research.google.com/github/LucasVCorrea/Data-Science/blob/main/trabajo_practico/ejercicio_21_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# El contenido con mayor cantidad de de acciones realizadas para todos los tipos posibles de acciones (⭐⭐⭐).

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=1ae5895c2a3ad5b2d5cec6b36636a6abb5edbf726aedf6d5af275cc74ab45837
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark
Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/

In [4]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd
import re as regex

In [5]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [6]:
rdd_logs = spark.read.csv('/content/drive/MyDrive/Organizacion De Datos/Trabajo_practico/logs.csv', header=True, inferSchema=True, multiLine=True, escape='"').rdd
#.select('item_id', 'action').rdd

In [9]:
rdd_logs.first()

Row(item_id=1, timestamp=datetime.datetime(2004, 12, 23, 7, 10, 9), contributor_username='Sabbut', contributor_id=2075, contributor_ip=None, comment="el contenido era: 'Si aprendés Pensamiento Científico ... TE VA S AENTERAR'", logtype='delete', action='delete', title='Verificacionismo')

In [25]:
rdd_logs_regex = rdd_logs.filter(lambda x:x.contributor_id!=None and x.action!=None and x.title!=None and ":" not in x.title)

In [26]:
# rdd_logs_regex.map(lambda x:x.action).distinct().collect()

['delete',
 'protect',
 'restore',
 'unprotect',
 'move',
 'move_redir',
 'renameuser',
 'modify',
 'patrol',
 'upload',
 'move_prot',
 'revision',
 'interwiki',
 'selfadd',
 'selfremove',
 'review',
 'skipbadns',
 'add',
 'remove',
 'unreview',
 'merge',
 'update',
 'delete_redir',
 'create',
 'addlink']

In [28]:
rdd_logs_regex_uso = rdd_logs_regex.map(lambda x:  ((x.title,x.action), 1)  ).reduceByKey(lambda a,b: a+b)

In [29]:
# rdd_logs_regex_uso.take(10)

[(('Pepe La Morcilla', 'delete'), 2),
 (('José Félix Esquivel', 'delete'), 1),
 (("L'Anse aux Meadows", 'delete'), 1),
 (('Eduardo Toba', 'delete'), 2),
 (('De que murio christell?', 'delete'), 1),
 (('Golden Latinoamerica', 'delete'), 1),
 (('Putón Verbenero', 'delete'), 1),
 (('Pepiuken', 'protect'), 1),
 (('Asamblea de cuba', 'delete'), 1),
 (('FERNANDO APARICIO BUENO', 'delete'), 1)]

In [30]:
rdd_logs_regex_uso.map(lambda x:( x[0][1], (x[1], x[0][0]) ) ).take(10)

[('delete', (4, 'Ambiplasma')),
 ('delete', (1, 'Gloria fuertes')),
 ('delete', (6, 'Bandera de la República Dominicana')),
 ('delete', (1, 'Juego de mesa')),
 ('delete', (2, 'Gobierno y política de República de Macedonia')),
 ('delete', (1, 'Campeche (ciudad)')),
 ('delete', (1, 'Los Molares (Sevilla)')),
 ('delete', (1, 'Tres Fuentes')),
 ('delete', (3, 'Manuela Sáenz')),
 ('delete', (6, 'José Antonio Correa'))]

In [31]:
rdd_final = rdd_logs_regex_uso.map(lambda x:( x[0][1], (x[1], x[0][0]) ) )

In [32]:
rdd_final.take(12)

[('delete', (2, 'Cómico')),
 ('delete', (2, 'Exodo')),
 ('delete', (2, 'Arganza (León)')),
 ('delete', (1, 'Hoplita')),
 ('delete', (2, 'Municipio de Ensenada')),
 ('delete', (1, 'Municipio de General Arenales')),
 ('delete', (1, 'Sega System')),
 ('delete', (1, 'Eritropoyesis')),
 ('delete', (2, 'Humillación sexual')),
 ('delete', (1, 'Elasmobranchii')),
 ('delete', (25, 'Alfabeto arameo')),
 ('delete', (3, 'Woodstock'))]

Respuesta: El contenido con más acciones realizadas para cada accion.

In [35]:
rdd_final.reduceByKey(lambda a,b: a if a[0] > b[0] else b).collect()

[('delete', (77, 'Agarradera (química)')),
 ('restore', (15, 'Abel Azcona')),
 ('move', (14, 'La Concordia (Nicaragua)')),
 ('move_redir', (15, 'Suspenso')),
 ('protect', (37, 'José de San Martín')),
 ('unprotect', (12, 'Cataluña')),
 ('patrol', (4, 'Yoshitoshi')),
 ('modify', (23, 'Cristina Fernández de Kirchner')),
 ('move_prot', (9, 'Rebelión en Libia de 2011')),
 ('revision', (43, 'Detención de Milagro Sala')),
 ('selfadd', (15, 'Ingeniería de software')),
 ('add', (57, '(commons)')),
 ('review', (13, 'Ingeniería de software')),
 ('selfremove', (3, 'Infraestructura (tecnología)')),
 ('remove', (3, 'Hot Pockets')),
 ('merge', (2, 'Amortiguación')),
 ('delete_redir', (11, 'Negre Lloma')),
 ('create', (12, 'Charlotte Caniggia')),
 ('update', (113, 'Bandera del Reino Unido')),
 ('addlink', (1, 'David Malet Armstrong')),
 ('unreview', (4, 'San Ángel')),
 ('interwiki', (2, 'Actinozoa')),
 ('skipbadns', (14, 'User Calypso')),
 ('upload', (2, "Théorème d'Al-Kashi")),
 ('renameuser', (1, 'M